In [0]:
#Por unica vez, eliminar!

dbutils.fs.rm('dbfs:/user/hive/warehouse/dim_time', recurse = True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/fact_orders', recurse = True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/dim_pizzas', recurse = True)

Out[1]: False

# PREGUNTAS OBJETIVO

* ¿Qué días y horas tendemos a estar más ocupados?
* ¿Cuántas pizzas estamos haciendo durante los períodos pico?
* ¿Cuáles son nuestras pizzas más vendidas y peores?
* ¿Cuál es nuestro valor promedio de pedido?
* ¿Qué tan bien estamos utilizando nuestra capacidad de asientos? (tenemos 15 mesas y 60 asientos)

## Catalogo de datos

Table	| Field	|Description
----------|------------|---------------
orders	|order_id|	Unique identifier for each order placed by a table
orders|	date|	Date the order was placed (entered into the system prior to cooking & serving)
orders	|time|	Time the order was placed (entered into the system prior to cooking & serving)
order_details|	order_details_id|	Unique identifier for each pizza placed within each order (pizzas of the same type and size are kept in the same row, and the quantity increases)
order_details|	order_id|	Foreign key that ties the details in each order to the order itself
order_details|	pizza_id|	Foreign key that ties the pizza ordered to its details, like size and price
order_details|	quantity|	Quantity ordered for each pizza of the same type and size
pizzas|	pizza_id|	Unique identifier for each pizza (constituted by its type and size)
pizzas|	pizza_type_id|	Foreign key that ties each pizza to its broader pizza type
pizzas|	size|	Size of the pizza (Small, Medium, Large, X Large, or XX Large)
pizzas|	price|	Price of the pizza in USD
pizza_types|	pizza_type_id|	Unique identifier for each pizza type
pizza_types|	name|	Name of the pizza as shown in the menu
pizza_types|	category|	Category that the pizza fall under in the menu (Classic, Chicken, Supreme, or Veggie)
pizza_types|	ingredients|	Comma-delimited ingredients used in the pizza as shown in the menu (they all include Mozzarella Cheese, even if not specified

# Modelado de nuestra tabla de hechos

Podemos identificar la tabla que registra las transacciones de pedidos (tabla orders). Para modelarla en el centro de nuestro esquema (intentaremos un star schema o snowflake schema), necesitamos que este conectada a las tablas que conformaran sus dimensiones. Para ello, primero la conectarmemos a la tabla detalles de ordenes (tabla order_details) por la columna order_id, clave primaria para nuestra tabla de ordenes y clave foranea para nuestra dimension de detalles de ordenes.

In [0]:
%sql
SELECT    ords.order_id, 
          ord_dts.order_details_id,
          ord_dts.pizza_id,
          ords.date,
          ords.time,
          ord_dts.quantity
       
FROM      orders               AS ords

LEFT JOIN order_details        AS ord_dts

ON        ords.order_id = ord_dts.order_id

order_id,order_details_id,pizza_id,date,time,quantity
1,1,hawaiian_m,2015-01-01,2023-02-03T11:38:36.000+0000,1
2,6,thai_ckn_l,2015-01-01,2023-02-03T11:57:40.000+0000,1
2,5,mexicana_m,2015-01-01,2023-02-03T11:57:40.000+0000,1
2,4,ital_supr_l,2015-01-01,2023-02-03T11:57:40.000+0000,1
2,3,five_cheese_l,2015-01-01,2023-02-03T11:57:40.000+0000,1
2,2,classic_dlx_m,2015-01-01,2023-02-03T11:57:40.000+0000,1
3,8,prsc_argla_l,2015-01-01,2023-02-03T12:12:28.000+0000,1
3,7,ital_supr_m,2015-01-01,2023-02-03T12:12:28.000+0000,1
4,9,ital_supr_m,2015-01-01,2023-02-03T12:16:31.000+0000,1
5,10,ital_supr_m,2015-01-01,2023-02-03T12:21:30.000+0000,1


Buen comienzo, tenemos nuestra tabla de hechos conectada a la dimension detalles de orden. Como sabemos algunas de nuestras consultas, realizaran filtros o agregaciones basadas en fechas, por lo que nos convendría **modelar al tiempo** como una dimensión.
(No es recomendable modelar pensando en consultas/reportes particulares, pero pensando que esta solución se va a mejorar, sirve como punto de partida)

# Modelado de la dimensión tiempo

In [0]:
%sql
SELECT    date,
          extract(YEAR FROM date) as year,
          extract(MONTH FROM date) as month,
          extract(DAY FROM date) as day,
          extract(HOUR FROM time) as hour 
       
FROM      orders 

ORDER BY  (date,hour)

date,year,month,day,hour
2015-01-01,2015,1,1,11
2015-01-01,2015,1,1,11
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,12
2015-01-01,2015,1,1,13


In [0]:
%sql
-- INTENTO QUEDARME CON SOLO 24hs (o las horas que registren pedidos) por cada fecha unica registrada . 

SELECT DISTINCT (date),
                extract(YEAR FROM date)  AS year,
                extract(MONTH FROM date) AS month ,
                extract(DAY FROM date)   AS day,
                extract(HOUR FROM time)  AS hour, 
                DAYOFWEEK(date)          AS day_week 
                
FROM            orders 

ORDER BY       (date,hour)

date,year,month,day,hour,day_week
2015-01-01,2015,1,1,11,5
2015-01-01,2015,1,1,12,5
2015-01-01,2015,1,1,13,5
2015-01-01,2015,1,1,14,5
2015-01-01,2015,1,1,15,5
2015-01-01,2015,1,1,16,5
2015-01-01,2015,1,1,17,5
2015-01-01,2015,1,1,18,5
2015-01-01,2015,1,1,19,5
2015-01-01,2015,1,1,20,5


In [0]:
%sql
CREATE OR REPLACE TABLE orders_temporal AS

    SELECT    ords.order_id, 
              ord_dts.order_details_id,
              ord_dts.pizza_id,
              ords.date,
              ords.time,
              ord_dts.quantity

    FROM      orders                   AS ords

    LEFT JOIN order_details            AS ord_dts

    ON        ords.order_id = ord_dts.order_id

num_affected_rows,num_inserted_rows


Bien, este esquema para nuestra dimención tiempo debería alcanzarnos para responer la preguntas objetivo contra nuestro modelo (totalmente mejorable).

Como los datos de fecha y hora (columnas date y time) estarán en mi dimensión, me gustaría quitarla de mi futura tabla de hechos, para que esta se relacione por una clave, contra la dimensión tiempo (ahorraremos espacio en nuestra tabla de hechos a toda costa). Para ello crearemos una clave incremental en esta ultima.

In [0]:
%sql
--creamos la tabla  time_temporal para insertar los registro dentro de la dimension tiempo. Luego se elimina
CREATE OR REPLACE TABLE time_temporal AS

  SELECT DISTINCT (date),
                  extract(YEAR FROM date)  AS year,
                  extract(MONTH FROM date) AS month ,
                  extract(DAY FROM date)   AS day,
                  extract(HOUR FROM time)  AS hour, 
                  DAYOFWEEK(date)          AS day_week 
                  
  FROM            orders 
  
  ORDER BY       (date,hour)

num_affected_rows,num_inserted_rows


In [0]:
%sql
--CREAMOS LA TABLA DIMENSION TIEMPO VACIA

CREATE TABLE  dim_time(
                      id BIGINT GENERATED BY DEFAULT AS IDENTITY,
                      date      DATE,
                      year      INT,
                      month     INT,
                      day       INT,
                      hour      INT,
                      day_week  INT
);


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-150530203227358> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-150530203227358> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFICBkaW1fdGltZSgKICAgICAgICAgICAgICAgICAgICAgIGlkIEJJR0lOVCBHRU5FUkFURUQgQlkgREVGQVVMVCBBUyBJREVOVElUWSwKICAgICAgICAgICAgICAgICAgICAgIGRhdGUgICAgICBEQVRFLAogICAgICAgICAgICAgICAgICAgICAgeWVhciAgICAgIElOVCwKICAgICAgICAgICAgICAgICAgICAgIG1vbnRoICAgICBJTlQsCiAgICAgICAgICAgICAgICAgICAgICBkYXkgICAgICAgSU5ULAogICAgICAgICAgICAgICAgICAgICAgaG91ciAgICAgIElOVCwKICAgICAgICAgICAgICAgICAgICAgIGRheV93ZWVrICBJTlQKKQ==").decode())
      5     display(df)
      6   

In [0]:
%sql
--INSERTAMOS VALORES EN LA DIMENSION TIEMPO PARA GENERAR LA CLAVE SUBROGADA (incremental) POR CADA REGISTRO
--https://www.databricks.com/blog/2022/08/08/identity-columns-to-generate-surrogate-keys-are-now-available-in-a-lakehouse-near-you.html
-- https://community.databricks.com/s/question/0D58Y00009COZfHSAX/how-to-add-an-identity-column-to-an-existing-table

INSERT INTO dim_time (
                 date,
                 year,
                 month,
                 day,
                 hour, 
                 day_week
)
                     
SELECT TRY_CAST( date     AS DATE),
       TRY_CAST( year     AS INT),
       TRY_CAST( month    AS INT),
       TRY_CAST( day      AS INT),
       TRY_CAST( hour     AS INT),
       TRY_CAST( day_week AS INT)
       
FROM time_temporal;

num_affected_rows,num_inserted_rows
4181,4181


In [0]:
%sql
--LA TABLA DE TIEMPO TEMPORAL CUMPLIÖ SU OBJETIVO, ASÏ QUE LA ELIMINAMOS

DROP TABLE time_temporal;

In [0]:
%sql
SELECT * FROM dim_time

id,date,year,month,day,hour,day_week
1,2015-01-01,2015,1,1,11,5
2,2015-01-01,2015,1,1,12,5
3,2015-01-01,2015,1,1,13,5
4,2015-01-01,2015,1,1,14,5
5,2015-01-01,2015,1,1,15,5
6,2015-01-01,2015,1,1,16,5
7,2015-01-01,2015,1,1,17,5
8,2015-01-01,2015,1,1,18,5
9,2015-01-01,2015,1,1,19,5
10,2015-01-01,2015,1,1,20,5


## Conección de la dim tiempo con la tabla orders

Dejaremos conectada nuestra tabla de órdenes por la clave primaria creada a la tabla de tiempo

In [0]:
%sql
SELECT * FROM orders_temporal LIMIT(1) 

order_id,order_details_id,pizza_id,date,time,quantity
1,1,hawaiian_m,2015-01-01,2023-02-03T11:38:36.000+0000,1


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/order_details',recurse=True)

In [0]:
#Union de la tabla orders a la dim_time

In [0]:
%sql
CREATE OR REPLACE TABLE orders_temporal AS

  SELECT order_id, 
         order_details_id, 
         pizza_id, 
         dim_t.id as dim_time_id, 
         quantity 

  FROM orders_temporal as ord

  LEFT JOIN dim_time   as dim_t

  ON        ord.date = dim_t.date

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM orders_temporal LIMIT(2)

order_id,order_details_id,pizza_id,dim_time_id,quantity
1,1,hawaiian_m,12,1
1,1,hawaiian_m,11,1


# Creacion de la tabla de hechos order

In [0]:
#----------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1992908430815321> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-1992908430815321> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("SU5TRVJUIElOVE8gZmFjdF9vcmRlcnMgKAogICAgICAgICAgICAgICAgICAgb3JkZXJfaWQsIAogICAgICAgICAgICAgICAgICAgcGl6emFfaWQsICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgIGRpbV90aW1lX2lkLCAgICAgICAgICAKICAgICAgICAgICAgICAgICAgIHF1YW50aXR5ICAgICAgICAgICAgIAopCiAgClNFTEVDVCAKICAgICAgICAgVFJZX0NBU1QoIG9yZGVyX2lkICAgICAgICAgQVMgSU5UKSwgICAgICAgICAKICAgICAgICAgVFJZX0NBU1QoIHBpenphX2lkICAgICAgICAgQVMgSU5UKSwKICAgICAgICAgVFJZX0NBU1QoIGRpbV90aW1lX2lkICAgICAgQVMgSU5UKSwKICAgICAgICAgVFJZX

# Modelado de la  dimensión Pizzas

In [0]:
%sql
SELECT * FROM pizzas

pizza_id,pizza_type_id,size,price
bbq_ckn_s,bbq_ckn,S,12.75
bbq_ckn_m,bbq_ckn,M,16.75
bbq_ckn_l,bbq_ckn,L,20.75
cali_ckn_s,cali_ckn,S,12.75
cali_ckn_m,cali_ckn,M,16.75
cali_ckn_l,cali_ckn,L,20.75
ckn_alfredo_s,ckn_alfredo,S,12.75
ckn_alfredo_m,ckn_alfredo,M,16.75
ckn_alfredo_l,ckn_alfredo,L,20.75
ckn_pesto_s,ckn_pesto,S,12.75


In [0]:
%sql
SELECT * FROM pizza_types

pizza_type_id,name,category,ingredients
bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno Peppers, Fontina Cheese, Gouda Cheese"
ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, Asiago Cheese, Alfredo Sauce"
ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garlic, Pesto Sauce"
southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Jalapeno Peppers, Corn, Cilantro, Chipotle Sauce"
thai_ckn,The Thai Chicken Pizza,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce"
big_meat,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
classic_dlx,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon"
hawaiian,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
ital_cpcllo,The Italian Capocollo Pizza,Classic,"Capocollo, Red Peppers, Tomatoes, Goat Cheese, Garlic, Oregano"


In [0]:
%sql
CREATE OR REPLACE TABLE pizzas_temporal AS

    SELECT       DISTINCT(pizza_id), 
                          pzz.pizza_type_id, 
                          pzz.size, 
                          pzz.price, 
                          pzz_t.name, 
                          pzz_t.category, 
                          pzz_t.ingredients

    FROM                  pizzas        AS pzz

    LEFT JOIN             pizza_types   AS pzz_t

    ON                    pzz.pizza_type_id = pzz_t.pizza_type_id

    ORDER BY              pizza_id      ASC

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM pizzas_temporal


pizza_id,pizza_type_id,size,price,name,category,ingredients
bbq_ckn_l,bbq_ckn,L,20.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
bbq_ckn_m,bbq_ckn,M,16.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
bbq_ckn_s,bbq_ckn,S,12.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
big_meat_l,big_meat,L,20.5,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
big_meat_m,big_meat,M,16.0,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
big_meat_s,big_meat,S,12.0,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
brie_carre_s,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Onions, Pears, Thyme, Garlic"
calabrese_l,calabrese,L,20.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
calabrese_m,calabrese,M,16.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
calabrese_s,calabrese,S,12.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"


In [0]:
%sql
SELECT * FROM orders_temporal 

order_id,order_details_id,pizza_id,dim_time_id,quantity
1,1,hawaiian_m,12,1
1,1,hawaiian_m,11,1
1,1,hawaiian_m,10,1
1,1,hawaiian_m,9,1
1,1,hawaiian_m,8,1
1,1,hawaiian_m,7,1
1,1,hawaiian_m,6,1
1,1,hawaiian_m,5,1
1,1,hawaiian_m,4,1
1,1,hawaiian_m,3,1


In [0]:
%sql
SELECT * FROM dim_pizzas 

id,pizza_id,pizza_type_id,size,name,category,ingredients
1,bbq_ckn_l,bbq_ckn,L,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
2,bbq_ckn_m,bbq_ckn,M,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
3,bbq_ckn_s,bbq_ckn,S,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
4,big_meat_l,big_meat,L,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
5,big_meat_m,big_meat,M,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
6,big_meat_s,big_meat,S,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
7,brie_carre_s,brie_carre,S,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Onions, Pears, Thyme, Garlic"
8,calabrese_l,calabrese,L,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
9,calabrese_m,calabrese,M,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
10,calabrese_s,calabrese,S,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"


In [0]:
%sql
SELECT   ords.order_id,
         ords.order_details_id,
         pzz.pizza_id,
         ords.dim_time_id,
         ords.quantity,
         pzz.pizza_type_id,
         pzz.size,
         pzz.price,
         pzz.name,
         pzz.category,
         pzz.ingredients
  FROM orders_temporal AS ords
  LEFT JOIN pizzas_temporal AS pzz
  ON ords.pizza_id = pzz.pizza_id

order_id,order_details_id,pizza_id,dim_time_id,quantity,pizza_type_id,size,price,name,category,ingredients
1,1,hawaiian_m,12,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,11,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,10,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,9,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,8,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,7,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,6,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,5,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,4,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,3,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"


In [0]:
%sql
CREATE OR REPLACE TABLE orders_temporal AS
  SELECT   ords.order_id,
         ords.order_details_id,
         pzz.pizza_id,
         ords.dim_time_id,
         ords.quantity,
         pzz.pizza_type_id,
         pzz.size,
         pzz.price,
         pzz.name,
         pzz.category,
         pzz.ingredients
  FROM orders_temporal AS ords
  LEFT JOIN pizzas_temporal AS pzz
  ON ords.pizza_id = pzz.pizza_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM orders_temporal

order_id,order_details_id,pizza_id,dim_time_id,quantity,pizza_type_id,size,price,name,category,ingredients
1,1,hawaiian_m,12,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,11,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,10,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,9,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,8,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,7,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,6,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,5,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,4,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,1,hawaiian_m,3,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"


In [0]:
%sql
CREATE OR REPLACE TABLE orders_temporal AS
  SELECT   ords.order_id,
           ords.dim_time_id,
           pzz.id        AS dim_pizzas_id,
           ords.quantity,
           ords.price

    FROM orders_temporal AS ords
    LEFT JOIN dim_pizzas AS pzz
    ON ords.pizza_id = pzz.pizza_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM dim_pizzas

id,pizza_id,pizza_type_id,size,name,category,ingredients
1,bbq_ckn_l,bbq_ckn,L,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
2,bbq_ckn_m,bbq_ckn,M,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
3,bbq_ckn_s,bbq_ckn,S,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
4,big_meat_l,big_meat,L,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
5,big_meat_m,big_meat,M,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
6,big_meat_s,big_meat,S,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
7,brie_carre_s,brie_carre,S,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Onions, Pears, Thyme, Garlic"
8,calabrese_l,calabrese,L,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
9,calabrese_m,calabrese,M,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
10,calabrese_s,calabrese,S,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"


num_affected_rows,num_inserted_rows


# Creacion de la tabla de hechos order

In [0]:
%sql

--CREAMOS LA TABLA DE HECHOS FACT_ORDERS VACÍA

CREATE TABLE  fact_orders(
                        id BIGINT GENERATED BY DEFAULT AS IDENTITY,
                        order_id             INT,
                        dim_time_id          BIGINT,
                        dim_pizzas_id        BIGINT,
                        quantity             INT,
                        price                DOUBLE
);


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2073772140327143> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-2073772140327143> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFICBmYWN0X29yZGVycygKICAgICAgICAgICAgICAgICAgICAgICAgaWQgQklHSU5UIEdFTkVSQVRFRCBCWSBERUZBVUxUIEFTIElERU5USVRZLAogICAgICAgICAgICAgICAgICAgICAgICBvcmRlcl9pZCAgICAgICAgICAgICBJTlQsCiAgICAgICAgICAgICAgICAgICAgICAgIGRpbV90aW1lX2lkICAgICAgICAgIEJJR0lOVCwKICAgICAgICAgICAgICAgICAgICAgICAgZGltX3Bpenphc19pZCAgICAgICAgQklHSU5ULAogICAgICAgICAgICAgICAgICAgICAgICBxdWFudGl0eSAgICAgICAgICAgICBJTlQsCiAgICAgICAgICAgICAgICAgICAgICAgIHByaWNlICAgICAgICAgICAgICAgIERPV

In [0]:
%sql
INSERT INTO fact_orders (
                  order_id,     
                  dim_time_id,  
                  dim_pizzas_id,
                  quantity,     
                  price        
)

SELECT 
         TRY_CAST( order_id            AS INT),         
         TRY_CAST( dim_time_id         AS BIGINT),
         TRY_CAST( dim_pizzas_id       AS BIGINT),
         TRY_CAST( quantity            AS INT),
         TRY_CAST( price               AS DOUBLE)
         
FROM orders_temporal;

num_affected_rows,num_inserted_rows
569559,569559


In [0]:
%sql
SELECT * FROM fact_orders

id,order_id,dim_time_id,dim_pizzas_id,quantity,price
1,1,12,33,1,13.25
2,1,11,33,1,13.25
3,1,10,33,1,13.25
4,1,9,33,1,13.25
5,1,8,33,1,13.25
6,1,7,33,1,13.25
7,1,6,33,1,13.25
8,1,5,33,1,13.25
9,1,4,33,1,13.25
10,1,3,33,1,13.25


In [0]:
%sql
SELECT * FROM dim_pizzas

id,pizza_id,pizza_type_id,size,name,category,ingredients
1,bbq_ckn_l,bbq_ckn,L,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
2,bbq_ckn_m,bbq_ckn,M,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
3,bbq_ckn_s,bbq_ckn,S,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
4,big_meat_l,big_meat,L,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
5,big_meat_m,big_meat,M,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
6,big_meat_s,big_meat,S,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
7,brie_carre_s,brie_carre,S,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Onions, Pears, Thyme, Garlic"
8,calabrese_l,calabrese,L,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
9,calabrese_m,calabrese,M,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
10,calabrese_s,calabrese,S,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"


In [0]:
%sql
--CREATE OR REPLACE TABLE fact_orders AS
  SELECT ords.order_id, 
         ords_dtls.order_details_id, 
         ords_dtls.pizza_id, 
         ords_dtls.quantity, 
         ords.date, 
         date_format(ords.time,'HH:mm:ss') as time
  FROM                orders        as ords
  LEFT JOIN           order_details as ords_dtls
  ON                  ords.order_id == ords_dtls.order_id
  

¿Qué días y horas tendemos a estar más ocupados?

In [0]:
%sql
SELECT         COUNT(order_id) as cant_orders,date, DAYOFWEEK( date) as day_week 
FROM           fact_orders 
GROUP BY       date 
ORDER BY       (cant_orders) DESC
LIMIT (30)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3811380183867452> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-3811380183867452> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("U0VMRUNUICAgICAgICAgQ09VTlQob3JkZXJfaWQpIGFzIGNhbnRfb3JkZXJzLGRhdGUsIERBWU9GV0VFSyggZGF0ZSkgYXMgZGF5X3dlZWsgCkZST00gICAgICAgICAgIGZhY3Rfb3JkZXJzIApHUk9VUCBCWSAgICAgICBkYXRlIApPUkRFUiBCWSAgICAgICAoY2FudF9vcmRlcnMpIERFU0MKTElNSVQgKDMwKQ==").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48   

¿Cuántas pizzas estamos haciendo durante los períodos pico?